In [ ]:
import pixeltable as pxt

tools = pxt.func.mcp.mcp_udfs(
    "http://localhost:9090/mcp",
)

process_video_tool = tools[0]
pxt.drop_dir("rick", force=True)
pxt.create_dir("rick", if_exists="replace_force")

tbl = pxt.create_table(
    path_str="rick.table", schema={"video_path": pxt.type_system.String, "video_name": pxt.type_system.String}
)

tbl.add_computed_column(
    video=process_video_tool(
        video_path=tbl.video_path,
    ),
    if_exists="replace_force",
)

tbl.insert(
    [
        {
            "video_path": "/app/videos/butter.mp4",
            "video_name": "rick-and-morty.mp4",
        }
    ]
)


In [12]:
import pixeltable as pxt

tbl = pxt.get_table("rick.table")

tools = pxt.func.mcp.mcp_udfs(
    "http://localhost:9090/mcp",
)

for tool in tools:
    print(f"Tool: {tool.name}")
    print(f"Signature: {tool.signature}")
    print()

tbl.add_column(user_query=pxt.type_system.String)

Tool: process_video
Signature: (video_path: String) -> String

Tool: get_video_clip_from_user_query
Signature: (video_path: String, user_query: String) -> String

Tool: get_video_clip_from_image
Signature: (video_path: String, user_image: String) -> String

Tool: ask_question_about_video
Signature: (video_path: String, user_query: String) -> String

Added 1 column value with 0 errors.


UpdateStatus(num_rows=1, num_computed_values=1, num_excs=0, updated_cols=[], cols_with_excs=[])

## Adding a MCP Tool as UDF

This block will add `get_clip_by_user_query` mcp tool, which is the second tool in the list, as a computed column to a table.

Observations:

- The pixeltable method to get the MCP tools, seems to work but is kind of over the hand. Because...
- 1. I've defined a table called rick.table where i've added a "video_name", "video_path" schema.
- 2. I added the `process_video` tool as a computed column in the table.
- 3. When i insert a video into the `video_path` column, it triggers the MCP tool that processes the video.
- 4. Now, If i want to add other tools, I have added a empty `user_query` column into the existing table.
- 5. I've added the `get_clip_by_user_query` MCP tool as a computed column into the table.
- **When inserting a new pair** <video_path>, <user_query> into the table the request gets sent to the MCP and it is processed, but, this is important, "The `process_video` also gets called one more time, because not only the user query column's got a new value, but "video_path" also.

Important #2
From the tools descriptions in MCP:

```
Tool: get_video_clip_from_user_query
Signature: (video_path: String, user_query: String) -> String

Tool: get_video_clip_from_image
Signature: (video_path: String, user_image: String) -> String

Tool: ask_question_about_video
Signature: (video_path: String, user_query: String) -> String
```

The signature of `get_video_clip_from_user_query` and `ask_question_about_video` are the same, "video_path", and "user_query". If we create a table with these tools, once we "call" a tool, every other computed column (a.k.a MCP tool) in the table will get triggered also.

This is good for pixeltable versioning system, but DOESN"T WORK for what we planned to build. Our Agent should select which tool to call and when.


In [ ]:
tbl.add_computed_column(
    clip_by_speech=tools[1](
        video_path=tbl.video_path,
        user_query=tbl.user_query,
    ),
    if_exists="replace_force",
)


tbl.add_computed_column(
    clip_by_speech=tools[1](
        video_path=tbl.video_path,
        user_query=tbl.user_query,
    ),
    if_exists="replace_force",
)

In [ ]:
tbl.insert([{"user_query": "pass the butter", "video_path": "/app/videos/butter.mp4"}])

Inserting rows into `table`: 1 rows [00:00, 457.24 rows/s]
Inserted 1 row with 0 errors.


UpdateStatus(num_rows=1, num_computed_values=7, num_excs=0, updated_cols=[], cols_with_excs=[])

In [ ]:
tbl.show()

video_path,video_name,video,user_query,clip_by_speech
/app/videos/butter.mp4,rick-and-morty.mp4,true,None,None
None,None,None,pass the butter,None
/app/videos/butter.mp4,None,false,pass the butter,"{ ""clip_path"": ""./videos/e9161d3f-cfb9-47ca-bbb9-bd20e3648d4e.mp4"" }"
